# HyporheicTools YAML Driver (Notebook Wrapper)

This notebook wraps the CLI/ArcGIS driver in src/HyporheicTools/core/run_from_yaml.py. Edit the configuration cell, run the driver, and inspect its return values/logs. All workflow logic continues to live in the Python package so maintenance stays centralized.


## Usage Notes

- Run inside the HyporheicTools/ArcGIS Python environment.
- The setup cell adds src/ to sys.path and enables autoreload so edits to the Python modules appear without restarting the kernel.
- Point yaml_path at your configuration file; optional flags mirror the CLI parameters.
- ArcGIS-specific behavior (group layers, contour building, etc.) remains inside 
un_from_yaml and is already guarded, so the notebook can run in environments without ArcGIS (warnings will print instead).


In [10]:
from __future__ import annotations

import sys
from pathlib import Path

from IPython import get_ipython


def _ensure_src_on_sys_path() -> Path:
    '''Ensure the repository's src directory is importable.'''
    here = Path.cwd().resolve()
    for base in [here, *here.parents]:
        src_candidate = base / "src"
        if (src_candidate / "HyporheicTools").exists():
            if str(src_candidate) not in sys.path:
                sys.path.insert(0, str(src_candidate))
            return src_candidate
    raise RuntimeError("Could not locate src/HyporheicTools. Launch the notebook from the project root or install the package.")


SRC_DIR = _ensure_src_on_sys_path()

ip = get_ipython()
if ip is not None:
    ip.run_line_magic("load_ext", "autoreload")
    ip.run_line_magic("autoreload", "2")

SRC_DIR


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


WindowsPath('C:/Users/gtmen/Desktop/HyporheicTools/src')

In [11]:
import importlib
from datetime import datetime
from pprint import pprint

from IPython.display import Markdown, display, Image

import HyporheicTools.core.run_from_yaml as rfy

importlib.reload(rfy)
run_from_yaml = rfy.run_from_yaml


In [12]:
# --- Configuration ---
from pathlib import Path

yaml_path = SRC_DIR.parent / "examples" / "basic_case" / "input" / "inputs.yaml"
out_folder = None  # Optional override for the output directory
make_figures = True  # Generate the figure suite
build_contours_in_driver = False  # Force contour build (True/False) or leave None to use YAML
dry_run = False  # Skip model build/run when True

if not yaml_path.exists():
    raise FileNotFoundError(f"Expected YAML at {yaml_path}")


In [13]:
# --- Execute the driver ---
import importlib
from datetime import datetime
from pprint import pprint
from IPython.display import Markdown, display

log_messages = []


def notebook_log(message: str) -> None:
    timestamped = f"[{datetime.now():%H:%M:%S}] {message}"
    log_messages.append(timestamped)
    print(timestamped)


run_args = dict(
    yaml_path=yaml_path,
    out_folder=out_folder,
    log=notebook_log,
    dry_run=dry_run,
    make_figures=make_figures,
    build_contours_in_driver=build_contours_in_driver,
)

filtered_args = {
    key: value
    for key, value in run_args.items()
    if value is not None or key in {"yaml_path", "log", "dry_run", "make_figures"}
}

importlib.reload(rfy)
run_from_yaml = rfy.run_from_yaml

result = run_from_yaml(**filtered_args)

display(Markdown("**run_from_yaml returned:**"))
pprint(result)


[02:03:06] Loading configuration from C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\input\inputs.yaml …
[02:03:06] Using packaged MODFLOW bin: C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\bin\modflow
[02:03:06] Workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model
[02:03:06] GWF workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model\gwf_workspace
[02:03:06] MP7 workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model\mp7_workspace
[02:03:06] STEP 1 — Preprocessing rasters/vectors …
Loaded HEC‑RAS CRS: PROJCS["NAD_1983_StatePlane_Texas_Central_FIPS_4203_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2296583.33333333],PARAMETER["False_Northing",9842500.0],PARAMETER["Central_Meridian",-100.333333333

C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\functions\my_utils.py:140: UserWarning: Warning: converting a masked element to nan.
  top[r, c] = terrain_elevation[row, col]


[02:03:09] STEP 3 — Defining boundaries & active domain …
[02:03:11] STEP 4 — Computing boundary heads from WSE edge + gradients …
[02:03:11]   Upstream-Left: distance to WSE-edge = 76.614, edge WSE = 1604.768
[02:03:11]   Upstream-Right: distance to WSE-edge = 61.067, edge WSE = 1604.766
[02:03:11]   Downstream-Left: distance to WSE-edge = 91.995, edge WSE = 1600.770
[02:03:11]   Downstream-Right: distance to WSE-edge = 145.166, edge WSE = 1601.240
[02:03:36] CHD cells prepared: 6978 (unique=6978, dupes=21)
[02:03:36] STEP 5 – Building & running models (MF6 + MP7) …
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model gwf_model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package chd_river...
    writing package chd_sides...
    writing package oc...
Running MODFLOW 6...
Running simulation: Hyporheic
FloPy is usin

C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\functions\my_utils.py:2096: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sqrt(np.diff(g["x"])**2 + np.diff(g["y"])**2).sum())
C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\functions\my_utils.py:2484: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lines_gdf_2d.to_file(l_shp, driver="ESRI Shapefile")
C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\functions\my_utils.py:2492: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lines_gdf_3d.to_file(shp3d, driver="ESRI Shapefile")
C:\Users\gtmen\Deskto

[02:04:22] 
[02:04:22] === Publication‑Ready Pathline Statistics ===
[02:04:22] Zone Budget — Hyporheic Throughflow (ft³/d)
[02:04:22]   budget file: gwf_model.cbb
[02:04:22]   term: CHD
[02:04:22]   kstpkper: (0, 0)
[02:04:22]   start cells: 46
[02:04:22]   end cells: 19
[02:04:22]   total start flow (signed): 943.725 ft³/d
[02:04:22]   total end flow (signed):   -1,493.738 ft³/d
[02:04:22]   |start|: 1,117.877 ft³/d
[02:04:22]   |end|:   1,493.738 ft³/d
[02:04:22]   hyporheic throughflow (avg(|start|,|end|)): 1,493.738 ft³/d
[02:04:22]   net flow balance (start+end): -550.013 ft³/d
[02:04:22] 
[02:04:22] Particle Path — 3D Length (ft)
[02:04:22]   count: 39
[02:04:22]   mean: 168.122 ft
[02:04:22]   median: 166.844 ft
[02:04:22]   min: 31.982 ft
[02:04:22]   max: 270.611 ft
[02:04:22]   p10: 114.882 ft
[02:04:22]   p90: 229.252 ft
[02:04:22] 
[02:04:22] Plan‑View Excursion (ft)
[02:04:22]   count: 39
[02:04:22]   mean: 158.235 ft
[02:04:22]   median: 157.631 ft
[02:04:22]   min: 30.9

**run_from_yaml returned:**

{'contours': {},
 'group_name': 'Hyporheic Results 2025-09-16 02:04',
 'head': {'geotiffs': ['C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L01.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L02.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L03.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L04.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L05.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\basic_case\\output\\summary\\head\\per_layer_tif\\head_L06.tif',
                       'C:\\Users\\gtmen\\Desktop\\HyporheicTools\\examples\\b

In [14]:
if log_messages:
    display(Markdown("**Logged messages**"))
    for entry in log_messages:
        print(entry)
else:
    print("No log messages recorded.")


**Logged messages**

[02:03:06] Loading configuration from C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\input\inputs.yaml …
[02:03:06] Using packaged MODFLOW bin: C:\Users\gtmen\Desktop\HyporheicTools\src\HyporheicTools\bin\modflow
[02:03:06] Workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model
[02:03:06] GWF workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model\gwf_workspace
[02:03:06] MP7 workspace: C:\Users\gtmen\Desktop\HyporheicTools\examples\basic_case\output\model\mp7_workspace
[02:03:06] STEP 1 — Preprocessing rasters/vectors …
[02:03:07] STEP 2 — Building model domain …
[02:03:09] STEP 3 — Defining boundaries & active domain …
[02:03:11] STEP 4 — Computing boundary heads from WSE edge + gradients …
[02:03:11]   Upstream-Left: distance to WSE-edge = 76.614, edge WSE = 1604.768
[02:03:11]   Upstream-Right: distance to WSE-edge = 61.067, edge WSE = 1604.766
[02:03:11]   Downstream-Left: distance to WSE-edge = 91.995, edge WSE = 160

In [15]:
pngs = []

if isinstance(result, dict):
    head_info = result.get("head")
    if isinstance(head_info, dict):
        pngs = head_info.get("pngs") or []
    if not pngs:
        pngs = result.get("pngs") or []

if pngs:
    display(Markdown("**Preview of generated PNGs**"))
    for png in pngs:
        try:
            display(Image(filename=str(png)))
        except Exception as exc:
            print(f"[WARN] Could not display {png}: {exc}")
else:
    print("Driver did not report any PNG outputs.")


Driver did not report any PNG outputs.
